In [1]:
! pip install bs4
import urllib.request
import requests
import numpy as np
from urllib.request import Request, urlopen
import pandas as pd
import re
from bs4 import BeautifulSoup
from random import randint
import time
import math
import csv
import string

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
#Parsing webpage
url_front = "https://www.allforyou.sg/"
req = Request(url_front, headers={'User-Agent': 'Chrome//5.0'})
page = urlopen(req)
soup = BeautifulSoup(page)
all_left_menu_item = soup.find('div', {'id' : 'a4u-cdd-content'})
broad_categories = all_left_menu_item.findAll('a', {'href' : '#'})
for i in range(0, len(broad_categories)):
    broad_categories[i] = broad_categories[i].get_text().rstrip().lstrip()
    broad_categories[i] = broad_categories[i].replace(", ", "-")
    broad_categories[i] = broad_categories[i].replace(" & ", "-")
    broad_categories[i] = broad_categories[i].replace(" ", "-")
    broad_categories[i] = broad_categories[i].replace(".", "")
    broad_categories[i] = broad_categories[i].replace("'", "")
    broad_categories[i] = url_front + broad_categories[i]
del broad_categories[0]
print(broad_categories)



['https://www.allforyou.sg/Fruits', 'https://www.allforyou.sg/Vegetables', 'https://www.allforyou.sg/Breakfast-Dairy', 'https://www.allforyou.sg/Meat-Poultry-Seafood', 'https://www.allforyou.sg/Rice-Noodles-Oil', 'https://www.allforyou.sg/Beverages', 'https://www.allforyou.sg/Canned-Food-Instant-Food', 'https://www.allforyou.sg/Chilled-Convenience', 'https://www.allforyou.sg/Frozen-Convenience', 'https://www.allforyou.sg/Condiments', 'https://www.allforyou.sg/Dried-Food-Baking', 'https://www.allforyou.sg/Snacks-Confectioneries', 'https://www.allforyou.sg/Baby-Products', 'https://www.allforyou.sg/Personal-Care', 'https://www.allforyou.sg/Household-Items', 'https://www.allforyou.sg/Health-Wellness', 'https://www.allforyou.sg/Other-Merchandise', 'https://www.allforyou.sg/Cartons-Products']


In [24]:
#Parsing to get narrower categories
complete_narrow_category = []
for i in range(0, len(broad_categories)):
    req = Request(broad_categories[i], headers={'User-Agent': 'Chrome//5.0'})
    page = urlopen(req)
    soup = BeautifulSoup(page)
    headers = soup.findAll('div', {'class' : 'FeaturedHeader'})
    for x in range(0,len(headers)):
        headers[x] = str(headers[x])
        headers[x] = re.findall(r'<a href="(.*?)" style', headers[x])[0]
        headers[x] = url_front[0:-1] + headers[x]
    complete_narrow_category.append(headers)
    print(i)
    
# # Trying to put page number into the link
list_with_page = list()
for i in range(0,len(complete_narrow_category)):
    list_to_append = list()
    for x in range(0,len(complete_narrow_category[i])):
        for z in range(1,4):
            link = complete_narrow_category[i][x] + '?pagenumber=' + str(z)
            list_to_append.append(link)
    list_with_page.append(list_to_append)
complete_narrow_category = list_with_page
# print(complete_narrow_category)
#     all_left_menu_item = soup.find('div', {'id' : 'a4u-cdd-content'})
#     broad_categories = all_left_menu_item.findAll('a', {'href' : '#'})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [43]:
#Conducting actual webscraping
product_details = {}
df = pd.DataFrame()
for i in range(0,len(complete_narrow_category)):
    for x in range(0,len(complete_narrow_category[i])):
        req = Request(complete_narrow_category[i][x], headers={'User-Agent': 'Chrome//5.0'})
        page = urlopen(req)
        soup = BeautifulSoup(page)
        product_class = soup.findAll('div', {'class' : 'productbox thumbnail text-center'})
        for z in range(0,len(product_class)):

            #image
            try:
                image_link = re.findall(r'"background:url((.*?)) ', str(product_class[z]))[0][0].replace("('", "").replace("')", "")
                product_details['Image'] = image_link
            except:
                product_details['Image'] = 'No Image'
            
            #Brand and Product
            try:
                brand_product = product_class[z].find('span', {'itemprop' : 'name'}).get_text()
                product_details['Brand'] = brand_product
            except:
                product_details['Brand'] = 'Brandless'
            
            #Further Details
            try:
                product_test_1 = product_class[z].findAll('div', {'class' : 'prod-data'})
                prod_des = re.findall(r'&lt;br&gt;&lt;b&gt; (.*?) &lt;/b&gt;', str(product_test_1))
                prod_des_2 = re.findall(r' &lt;/b&gt; (.*?). &lt;', str(product_test_1))
                prod_des.append(prod_des_2[0])
                add_info = ', '.join(prod_des)
                product_details['Further Details'] = add_info
            except:
                try:
                    #try out if the product_des_2 in the initial try-except is the main problem
                    product_test_1 = product_class[z].findAll('div', {'class' : 'prod-data'})
                    prod_des = re.findall(r'&lt;br&gt;&lt;b&gt; (.*?) &lt;/b&gt;', str(product_test_1))
                    add_info = ', '.join(prod_des)
                    product_details['Further Details'] = add_info
                except:
                    try:
                        #try out if the product_des is the problem
                        product_test_1 = product_class[z].findAll('div', {'class' : 'prod-data'})
                        prod_des_2 = re.findall(r' &lt;/b&gt; (.*?). &lt;', str(product_test_1))
                        product_details['Further Details'] = prod_des_2[0]
                    except:
                        product_details['Further Details'] = 'No Further Details'
            if len(product_details['Further Details']) == 0:
                product_details['Further Details'] = 'No Further Details'
            
            #Current Price
            try:
                current_price = product_class[z].find('div', {'class' : 'price'}).get_text()
                product_details['Current Price'] = current_price
            except:
                product_details['Current Price'] = 'No Price'
            
            #Discounts & old price
            try:
                discount = product_class[z].find('span', {'itemprop' : 'offer'}).get_text()
                product_details['Discount'] = discount
                try:
                    old_price = re.findall(r'data-oldprice=(.*?) data', str(product_class[z]))[0][1:-1]
                    product_details['Old Price'] = old_price
                except:
                    product_details['Old Price'] = 'Look at new price and discount instead. There is discount on this item.'
            except:
                product_details['Discount'] = 'No Discount'
                product_details['Old Price'] = 'No Discount'
            if len(product_details['Discount']) == 1:
                product_details['Discount'] = str(int(((float(old_price[1:]) - float(current_price[1:]))/ float(old_price[1:]))* 100)) + '% off'
            if len(product_details['Old Price']) == 0:
                product_details['Old Price'] = 'Refer to Discount column for the discount when purchased in a certain quantity.'
            
            # Stock issue
            try:
                out_stock = re.findall(r'data-outofstock="(.*?)" data-price', str(product_class[z]))[0]
                if out_stock == 'True':
                    product_details['Stock'] = 'Out of Stock'
                else:
                    product_details['Stock'] = "In Stock"
            except:
                product_details['Stock'] = 'Error in determining stock'
            
            #Brand Goods Page
            product_details['Brands Goods'] = 'No page exist for Sheng Shiong'
            
            #Company
            product_details['Company'] = 'Sheng Shiong'
            
            #Size
            try:
                size = re.findall(r'span style="color: grey;"&gt;(.*?)&lt;/', str(product_class[z]))[0]
                alt_size = re.findall(r'&lt;b&gt;(.*?) &lt;/b&gt;&lt;/span&gt;', str(product_class[z]))[0]
                product_details['Size'] = size + alt_size
            except:
                try:
                    #testing if its the second line that caused the error
                    size = re.findall(r'span style="color: grey;"&gt;(.*?)&lt;/', str(product_class[z]))[0]
                    product_details['Size'] = size 
                except:
                    try:
                        #testing if its the third line that caused the error
                        alt_size = re.findall(r'&lt;b&gt;(.*?) &lt;/b&gt;&lt;/span&gt;', str(product_class[z]))[0]
                        product_details['Size'] = alt_size
                    except:
                        product_details['Size'] = 'No Size'      
            #Category Classification
            narrow_cat = re.findall(r'https://www.allforyou.sg/(.*?)?pagenumber', str(complete_narrow_category[i][x]))[0][0:-1]
            product_details['Narrow Category'] = narrow_cat
            product_details['Broad Cateogry'] = broad_categories[i].split('https://www.allforyou.sg/')[-1]
            df = df.append(product_details, ignore_index = True)
        print(x)
    print(i)
# print(df)
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
7
8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
1

In [113]:
product_class = soup.findAll('div', {'class' : 'productbox thumbnail text-center'})
# print(product_class[0])

#discount
print(product_class[0].find('span', {'itemprop' : 'offer'}).get_text())

#image link
image_link = re.findall(r'"background:url((.*?)) ', str(product_class[0]))[0][0].replace("('", "").replace("')", "")
print(image_link)

#Brand and product
print(product_class[0].find('span', {'itemprop' : 'name'}).get_text())

#Further details
product_test = soup.findAll('div', {'class' : 'productbox thumbnail text-center'})
product_test_1 = product_test[17].findAll('div', {'class' : 'prod-data'})
prod_des = re.findall(r'&lt;br&gt;&lt;b&gt; (.*?) &lt;/b&gt;', str(product_test_1))
prod_des_2 = re.findall(r' &lt;/b&gt; (.*?). &lt;', str(product_test_1))
prod_des.append(prod_des_2[0])
add_info = ', '.join(prod_des)
print(add_info)

#Current Price
print(product_class[0].find('div', {'class' : 'price'}).get_text())

#Old Price
old_price = re.findall(r'data-oldprice=(.*?) data', str(product_class[0]))[0][1:-1]
print(old_price)
#Discount without old price
##use dictionary length to direct people to look at dscount price instead

#instock or not
out_stock = re.findall(r'data-outofstock="(.*?)" data-price', str(product_test[2]))[0]
if out_stock == 'True':
    print('Out of Stock')
else:
    print('In Stock')
    
#Brand Goods
print('No page to consolidate brand goods')

#Company
print('Sheng Shiong')

#Size
size = re.findall(r'span style="color: grey;"&gt;(.*?)&lt;/', str(product_test[7]))[0]
print(size)
#alt size
alt_size = re.findall(r'&lt;b&gt;(.*?) &lt;/b&gt;&lt;/span&gt;', str(product_test[17]))[0]
print(alt_size)

5 for $3.99 
https://s3-ap-southeast-1.amazonaws.com/a4ucontent/images/0022304_125.png
CHN Fuji Apple 
$0.85



In [110]:
url_front = "https://www.allforyou.sg/baking-ingredients"
req = Request(url_front, headers={'User-Agent': 'Chrome//5.0'})
page = urlopen(req)
soup = BeautifulSoup(page)
product_test = soup.findAll('div', {'class' : 'productbox thumbnail text-center'})
print(product_test[39])
old_price = re.findall(r'data-oldprice=(.*?) data', str(product_test[39]))[0][1:-1]
print(old_price)

<div class="productbox thumbnail text-center" style="position: relative">
        <div id="TurnArund-4031" style="display:none;">0</div>
                <div class="selectedQnty" id="4031" style="display:none;" title="Total selected Quantity 0">0</div>
        
        <a class="prodnamelink" onclick="showProductDetailsPopup('/addproducttocart/','/Catalog/GetProductSelectedQuantity','#prod-data-id4031')">
                <span class="offerStickerOverlay"></span>

            <div class="thumb productbox-bgimg-div" id="productImageThumb" style="background:url('https://s3-ap-southeast-1.amazonaws.com/a4ucontent/images/0056095_125.png') no-repeat center center;">
            </div>
        </a>

        <div class="caption">
            <div class="prodname ">
                <a class="prodnamelink" onclick="showProductDetailsPopup('/addproducttocart/','/Catalog/GetProductSelectedQuantity','#prod-data-id4031')">
                    <span itemprop="name">Hershey's Semi Choco Chip 340G </sp

In [122]:
url_front = "https://www.allforyou.sg/wholegrain-glutinous-rice"
req = Request(url_front, headers={'User-Agent': 'Chrome//5.0'})
page = urlopen(req)
soup = BeautifulSoup(page)
product_test = soup.findAll('div', {'class' : 'productbox thumbnail text-center'})
# print(product_test[2])
out_stock = re.findall(r'data-outofstock="(.*?)" data-price', str(product_test[2]))[0]
if out_stock == 'True':
    print('Out of Stock')
else:
    print('In Stock')

out of stock


In [39]:
# #trial for old price
# dit = {}
# dit['LOL'] = 'price'
# if len(dit['LOL']) >= 1:
#     print('not empty')
print(len(product_details['Old Price']))

0


In [45]:
print(df)
df.to_csv('Sheng Shiong Scrape.csv', sep =',')

                                          Brand  \
0                               CHN Fuji Apple    
1                         N.Z Royal Gala Apple    
2                              S.A Green Apple    
3                          N.Z Kanzi Apple 1KG    
4                    CHN Premium Fuji Apple 4s    
5                             S.A Packham Pear    
6              CHN Premium Candy Fuji Apple 4s    
7                    N.Z Royal Gala Apple 900G    
8                          S.A Green Apple 1KG    
9                          N.Z Queen Apple 1KG    
10                          N.Z Modi Apple 1KG    
11                    S.A Red Candy Apple 500G    
12                          S.A Fuji Apple 1KG    
13                  USA Red Delicious Apple 5s    
14                 FRA Organic Juliet Apple 4s    
15                   S.A Red Forelle Pear 800G    
16                        CHN Nam Shui Pear 4S    
17         N.ZEALAND/BELGIUM Rockit Apple 5-6s    
18                            S